In [573]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
from sklearn import model_selection
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import timm
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader
import time
import copy

In [574]:


data = yf.download('EURJPY=X', start='2001-01-03', end='2023-03-09')

data.head()


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-01-23,126.980003,127.110001,126.519997,126.879997,126.879997,0
2003-01-24,126.900002,127.760002,126.800003,127.489998,127.489998,0
2003-01-27,127.589996,128.940002,127.489998,128.440002,128.440002,0
2003-01-28,128.460007,128.720001,128.020004,128.470001,128.470001,0
2003-01-29,128.460007,128.619995,127.889999,128.279999,128.279999,0


In [575]:
data['Up'] = 0
# future_num日後の終値の上下を予測
future_num = 1
for i in range(len(data)-future_num):
    if data['Close'][i] < data['Close'][i+future_num]:
        data['Up'][i] = 1

C:\Users\re6de\AppData\Local\Temp\ipykernel_34404\2024462940.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Up'][i] = 1
C:\Users\re6de\AppData\Local\Temp\ipykernel_34404\2024462940.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Up'][i] = 1
C:\Users\re6de\AppData\Local\Temp\ipykernel_34404\2024462940.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Up'][i] = 1
C:\Users\re6de\AppData\Local\Temp\ipykernel_34

In [576]:
data['ImageName'] =  ['./DataFolder2/' + str(data.index[i].year)+ '-'  + str(data.index[i].month) + '-' + str(data.index[i].day) + '.png' for i in range(len(data))]

In [577]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,Up,ImageName
Date,,,,,,,,
2023-03-02,145.292999,145.552002,144.809998,145.292999,145.292999,0,0,./DataFolder2/2023-3-2.png
2023-03-03,144.895996,145.051727,144.307999,144.895996,144.895996,0,0,./DataFolder2/2023-3-3.png
2023-03-06,144.524002,145.369995,144.132004,144.524002,144.524002,0,1,./DataFolder2/2023-3-6.png
2023-03-07,145.311005,145.432999,144.623993,145.311005,145.311005,0,0,./DataFolder2/2023-3-7.png
2023-03-08,144.917999,145.214996,144.244995,144.917999,144.917999,0,0,./DataFolder2/2023-3-8.png


In [578]:
img_create = 1
if img_create == 1:
    seq_len = 20

    data.fillna(0)
    #data.reset_index(inplace=True)
    data['Date2'] = data.index
    data['Date2'] = data['Date2'].map(mdates.date2num)
    for i in range(0, len(data)-1):
        c = data.iloc[i:i + int(seq_len) , :]
        if len(c) == int(seq_len):
            # Date,Open,High,Low,Adj Close,Volume
            target = ['Date2','Open','High','Low','Close']
            pngfile = data['ImageName'][i + int(seq_len)-2]

            my_dpi = 192
            # Create my own `marketcolors` to use with the `nightclouds` style:
            mc = mpf.make_marketcolors(up='#77d879',down='#db3f3f',inherit=True)

            # Create a new style based on `nightclouds` but with my own `marketcolors`:
            s  = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=mc,gridstyle = '')

            fig = mpf.figure(style=s,figsize=(96 / my_dpi, 96 / my_dpi), dpi=my_dpi)
            ax1 = fig.add_subplot(1,1,1)

            ax1.grid(False)
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.xaxis.set_visible(False)
            ax1.yaxis.set_visible(False)
            ax1.axis('off')

            mpf.plot(c[target], ax=ax1,volume=False,type='candle',update_width_config=dict(candle_linewidth=1))
            fig.savefig(pngfile,  pad_inches=0, transparent=False, bbox_inches="tight")

            plt.close(fig)


    print("Converting olhc to candlestik finished.")

Converting olhc to candlestik finished.


In [579]:
"""
img_create = 1
if img_create == 1:
    seq_len = 20

    data.fillna(0)
    #data.reset_index(inplace=True)
    data['Date2'] = data.index
    data['Date2'] = data['Date2'].map(mdates.date2num)
    for i in range(0, len(data)-1):
        c = data.iloc[i:i + int(seq_len) , :]
        if len(c) == int(seq_len):
            # Date,Open,High,Low,Adj Close,Volume
            target = ['Date2','Open','High','Low','Close']
            pngfile = data['ImageName'][i + int(seq_len)-2]

            my_dpi = 96
            # Create my own `marketcolors` to use with the `nightclouds` style:
            mc = mpf.make_marketcolors(up='#77d879',down='#db3f3f',inherit=True)

            # Create a new style based on `nightclouds` but with my own `marketcolors`:
            s  = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=mc,gridstyle = '')

            fig = mpf.figure(style=s,figsize=(48 / my_dpi, 48 / my_dpi), dpi=my_dpi)
            ax1 = fig.add_subplot(1,1,1)

            ax1.grid(False)
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.xaxis.set_visible(False)
            ax1.yaxis.set_visible(False)
            ax1.axis('off')

            mpf.plot(c[target], ax=ax1,volume=False,type='candle',update_width_config=dict(candle_linewidth=1))
            fig.savefig(pngfile,  pad_inches=0, transparent=False, bbox_inches="tight")

            plt.close(fig)


    print("Converting olhc to candlestik finished.")
"""

'\nimg_create = 1\nif img_create == 1:\n    seq_len = 20\n\n    data.fillna(0)\n    #data.reset_index(inplace=True)\n    data[\'Date2\'] = data.index\n    data[\'Date2\'] = data[\'Date2\'].map(mdates.date2num)\n    for i in range(0, len(data)-1):\n        c = data.iloc[i:i + int(seq_len) , :]\n        if len(c) == int(seq_len):\n            # Date,Open,High,Low,Adj Close,Volume\n            target = [\'Date2\',\'Open\',\'High\',\'Low\',\'Close\']\n            pngfile = data[\'ImageName\'][i + int(seq_len)-2]\n\n            my_dpi = 96\n            # Create my own `marketcolors` to use with the `nightclouds` style:\n            mc = mpf.make_marketcolors(up=\'#77d879\',down=\'#db3f3f\',inherit=True)\n\n            # Create a new style based on `nightclouds` but with my own `marketcolors`:\n            s  = mpf.make_mpf_style(base_mpf_style=\'nightclouds\',marketcolors=mc,gridstyle = \'\')\n\n            fig = mpf.figure(style=s,figsize=(48 / my_dpi, 48 / my_dpi), dpi=my_dpi)\n          

In [580]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(data['ImageName'].values, data['Up'].values, test_size=0.1,random_state=1)



In [581]:
X = []
Y = []
X_val = []
Y_val = []

transform = transforms.Compose([transforms.ToTensor()])

#for image_file,label in zip(x_train,y_train):
for image_file,label in zip(x_train,y_train):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        #image = np.asarray(image)
        X.append(image)
        Y.append(label)
    except FileNotFoundError as e:
        print(e)


for image_file,label in zip(x_val,y_val):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        X_val.append(image)
        Y_val.append(label)
    except FileNotFoundError as e:
        print(e)

[Errno 2] No such file or directory: './DataFolder2/2003-2-10.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-7.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-17.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-11.png'
[Errno 2] No such file or directory: './DataFolder2/2023-3-8.png'
[Errno 2] No such file or directory: './DataFolder2/2003-1-24.png'
[Errno 2] No such file or directory: './DataFolder2/2003-1-31.png'
[Errno 2] No such file or directory: './DataFolder2/2003-1-28.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-3.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-5.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-6.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-14.png'
[Errno 2] No such file or directory: './DataFolder2/2003-2-4.png'
[Errno 2] No such file or directory: './DataFolder2/2003-1-23.png'
[Errno 2] No such file or directory: './DataFolder2/2003-1-27.png'
[

In [582]:
import numpy as np

# NumPy配列として保存
np.save('./data/x_train.npy', x_train)
np.save('./data/x_val.npy', x_val)
np.save('./data/y_train.npy', y_train)
np.save('./data/y_val.npy', y_val)


In [583]:
class EUR_Dataset(Dataset):
    def __init__(self, image, label_list, phase=None):

        self.transform = transforms.Compose([transforms.ToTensor()])
        self.image = image
        self.label_list = torch.tensor(label_list,dtype=torch.long)
        self.phase = phase


    def __getitem__(self,index):
        # index番目の画像を読み込み、前処理を行う
        image = self.image[index]
        # index番目のラベルを取得する
        label = self.label_list[index]

        return image, label

    def __len__(self):
        return len(self.image)

In [584]:
train_dataset = EUR_Dataset(X,Y,phase='train')
total_samples = len(train_dataset)
print(total_samples)
val_dataset = EUR_Dataset(X_val,Y_val,phase='val')
val_samples = len(val_dataset)
print(val_samples)

4684
521


In [585]:

model = models.resnet18(pretrained=False)

num_ftrs = model.fc.in_features

model.fc  = nn.Sequential(
                      nn.Linear(num_ftrs ,2))

c:\Users\re6de\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\re6de\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [586]:
from torch.optim import lr_scheduler

In [587]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

criterion = nn.CrossEntropyLoss()


# Observe that all parameters are being optimized
optimizer_ft = optim.AdamW(model.parameters())  # 初期学習率を例として0.01に設定
num_epochs = 40
# LambdaLRの定義


batchsize = 64

loss_list = []
val_loss_list = []
# DataLoaderのインスタンス作成
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsize, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsize, shuffle=False)

# データローダーを辞書として格納する
dataloaders_dict = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

cuda:0


In [588]:
Train_loss = []
Val_loss = []
Train_acc = []
Val_acc = []


def train_model(model, criterion, optimizer,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    #best_acc = 0.0
    best_loss = 1000.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        lambda1 = lambda epoch: 0.1 if epoch >= num_epochs // 2 else 1.0  # num_epochsはトータルのエポック数
        scheduler = lr_scheduler.LambdaLR(optimizer_ft, lr_lambda=lambda1)
        # Each epoch has a training and validation phase

        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0

            # 学習モード、検証モードの切替
            if phase == 'train':
                model.train()
            else:
                model.eval()

            #for i,data in enumerate(trainloader):
            i = 0
            data_num = 0
            for inputs, labels in dataloaders_dict[phase]:
                i += 1
                #for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, axis=1)
                  
                    #print(preds)
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    data_num += inputs.size(0)

            epoch_loss = running_loss/data_num
            epoch_acc = running_corrects.double()/data_num


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'train':
                Train_loss.append(epoch_loss)
                Train_acc.append(epoch_acc)
            else:
                Val_loss.append(epoch_loss)
                Val_acc.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [589]:
model = train_model(model,criterion,optimizer_ft,num_epochs=num_epochs)

Epoch 0/39
----------
train Loss: 0.5435 Acc: 0.7301
val Loss: 0.4546 Acc: 0.7965

Epoch 1/39
----------
train Loss: 0.4139 Acc: 0.8224
val Loss: 0.3887 Acc: 0.8215

Epoch 2/39
----------
train Loss: 0.3614 Acc: 0.8388
val Loss: 0.3617 Acc: 0.8407

Epoch 3/39
----------
train Loss: 0.3089 Acc: 0.8599
val Loss: 0.3608 Acc: 0.8426

Epoch 4/39
----------
train Loss: 0.2773 Acc: 0.8747
val Loss: 0.3441 Acc: 0.8580

Epoch 5/39
----------
train Loss: 0.2472 Acc: 0.8896
val Loss: 0.3595 Acc: 0.8445

Epoch 6/39
----------
train Loss: 0.2667 Acc: 0.8798
val Loss: 0.3273 Acc: 0.8445

Epoch 7/39
----------
train Loss: 0.2073 Acc: 0.9093
val Loss: 0.3813 Acc: 0.8407

Epoch 8/39
----------
train Loss: 0.2153 Acc: 0.9065
val Loss: 0.3532 Acc: 0.8388

Epoch 9/39
----------
train Loss: 0.1782 Acc: 0.9238
val Loss: 0.3991 Acc: 0.8330

Epoch 10/39
----------
train Loss: 0.1662 Acc: 0.9291
val Loss: 0.4601 Acc: 0.8196

Epoch 11/39
----------
train Loss: 0.1250 Acc: 0.9498
val Loss: 0.5127 Acc: 0.8253

Ep

In [590]:
# 予測結果、予測確率を記録する

pred_list = np.empty([1, 2])

i = 0
input_list = []
output_list = []

for inputs, labels in val_dataloader:
    i += 1
    #for inputs, labels in dataloaders[phase]:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        sm = nn.Softmax(dim=1)
        outputs2 = np.array(sm(outputs.cpu()))
        pred_list = np.vstack((pred_list,outputs2))
        output_list.append(outputs2)
        input_list.append(labels)
      
i

9

In [591]:
input_list = torch.cat(input_list, dim=0).cpu().numpy()
len(input_list)

521

In [592]:
# 抽出した右の列を格納するリスト
right_columns = [array[:, 1] for array in output_list]
print(right_columns)
# 抽出した列を連結
one_predict = np.concatenate(right_columns)
print(one_predict)
# 抽出した右の列を格納するリスト
left_columns = [array[:, 0] for array in output_list]

# 抽出した列を連結
zero_predict = np.concatenate(left_columns)


[array([2.13484719e-01, 5.50112640e-03, 4.17509764e-05, 9.47279394e-01,
       9.47177351e-01, 5.17242014e-01, 7.12563097e-03, 4.73402534e-03,
       7.35161841e-01, 4.84587438e-02, 9.94486868e-01, 1.25862332e-03,
       7.49371648e-01, 1.35392562e-01, 7.50470102e-01, 9.85265911e-01,
       8.44842553e-01, 2.34726463e-02, 1.78754464e-01, 9.88973439e-01,
       2.73299236e-02, 5.92967987e-01, 4.16116893e-01, 9.90662754e-01,
       8.63903642e-01, 4.50942712e-03, 3.38177457e-02, 1.76207885e-01,
       9.31528926e-01, 9.84850287e-01, 4.32120748e-02, 6.96193576e-01,
       1.56541299e-02, 1.92424108e-03, 4.48657805e-03, 9.93150294e-01,
       5.47992513e-02, 9.63648319e-01, 3.50942649e-02, 1.74725994e-01,
       3.71646858e-03, 2.62933403e-01, 2.26654671e-03, 9.99586046e-01,
       9.31209087e-01, 3.26821767e-02, 4.38318342e-01, 1.21405837e-03,
       1.93568307e-03, 8.32780525e-02, 8.70231152e-01, 5.98305697e-03,
       1.51353717e-01, 5.47020793e-01, 5.03520574e-03, 9.87882912e-01,
     

In [593]:
one_predict

array([2.13484719e-01, 5.50112640e-03, 4.17509764e-05, 9.47279394e-01,
       9.47177351e-01, 5.17242014e-01, 7.12563097e-03, 4.73402534e-03,
       7.35161841e-01, 4.84587438e-02, 9.94486868e-01, 1.25862332e-03,
       7.49371648e-01, 1.35392562e-01, 7.50470102e-01, 9.85265911e-01,
       8.44842553e-01, 2.34726463e-02, 1.78754464e-01, 9.88973439e-01,
       2.73299236e-02, 5.92967987e-01, 4.16116893e-01, 9.90662754e-01,
       8.63903642e-01, 4.50942712e-03, 3.38177457e-02, 1.76207885e-01,
       9.31528926e-01, 9.84850287e-01, 4.32120748e-02, 6.96193576e-01,
       1.56541299e-02, 1.92424108e-03, 4.48657805e-03, 9.93150294e-01,
       5.47992513e-02, 9.63648319e-01, 3.50942649e-02, 1.74725994e-01,
       3.71646858e-03, 2.62933403e-01, 2.26654671e-03, 9.99586046e-01,
       9.31209087e-01, 3.26821767e-02, 4.38318342e-01, 1.21405837e-03,
       1.93568307e-03, 8.32780525e-02, 8.70231152e-01, 5.98305697e-03,
       1.51353717e-01, 5.47020793e-01, 5.03520574e-03, 9.87882912e-01,
      

In [598]:
#  閾値とする確率、上昇、下降のどちらを予想するか、閾値を設定した時の平均変動価格を出力する関数を作成

def print_ext(updown, rate,one_predict):
    total_counter = 0
    counter = 0
    change_price = 0

    if updown == 'UP':
        UPtarget = 1
   
    
    if UPtarget == 1:
        for i in range(len(input_list)-1):  # Adjust the range here
            if  one_predict[i] > rate:
                if input_list[i] == 1:
                    counter += 1 
    total_counter = input_list.sum()
            

    print('精度: {:4f}'.format(counter / total_counter))
    print('正解数: {:4f}'.format(counter))
    print('{}数: {:4f}'.format(updown, total_counter))


In [599]:
np.set_printoptions(threshold=np.inf)

In [601]:
print('-----閾値0.5精度-----')
print_ext('UP',0.5,one_predict)
print()
print('-----閾値0.8精度-----')
print_ext('UP',0.8,one_predict)
print()
print('-----閾値0.9精度-----')
print_ext('UP',0.9,one_predict)
print()

-----閾値0.5精度-----
精度: 0.864000
正解数: 216.000000
UP数: 250.000000

-----閾値0.8精度-----
精度: 0.728000
正解数: 182.000000
UP数: 250.000000

-----閾値0.9精度-----
精度: 0.612000
正解数: 153.000000
UP数: 250.000000

